In [1]:
import numpy as np
import scipy.sparse as sparse
from scipy.linalg import expm, norm
from scipy.io import mmread
from timeit import default_timer as timer


In [2]:
# create adjacency matrix as array

A_array = np.array([[0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]])
A_array

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1],
       [1, 0, 0, 1, 0],
       [0, 1, 1, 0, 1],
       [0, 1, 0, 1, 0]])

In [3]:
# create as sparse matrix

A = sparse.csc_matrix(A_array)
print(A)

  (2, 0)	1
  (3, 1)	1
  (4, 1)	1
  (0, 2)	1
  (3, 2)	1
  (1, 3)	1
  (2, 3)	1
  (4, 3)	1
  (1, 4)	1
  (3, 4)	1


In [4]:
# calculate exponential using A_csc

expA = expm(A)

print(expA)

  (0, 0)	1.5926873357152362
  (1, 0)	0.2656190865440594
  (2, 0)	1.3912135792878675
  (3, 0)	0.7107814780509464
  (4, 0)	0.26561908654405947
  (0, 1)	0.26561908654405947
  (1, 1)	2.78510028595431
  (2, 1)	0.9764005645950063
  (3, 1)	2.6332332304269346
  (4, 1)	2.417220844782868
  (0, 2)	1.3912135792878675
  (1, 2)	0.9764005645950061
  (2, 2)	2.303468813766183
  (3, 2)	1.922451752375987
  (4, 2)	0.9764005645950061
  (0, 3)	0.7107814780509464
  (1, 3)	2.6332332304269337
  (2, 3)	1.9224517523759868
  (3, 3)	3.545488464905249
  (4, 3)	2.6332332304269337
  (0, 4)	0.26561908654405936
  (1, 4)	2.4172208447828676
  (2, 4)	0.9764005645950061
  (3, 4)	2.633233230426934
  (4, 4)	2.7851002859543104


In [5]:
# put back into dense matrix
mtx_expA = expA.todense()
mtx_expA

matrix([[1.59268734, 0.26561909, 1.39121358, 0.71078148, 0.26561909],
        [0.26561909, 2.78510029, 0.97640056, 2.63323323, 2.41722084],
        [1.39121358, 0.97640056, 2.30346881, 1.92245175, 0.97640056],
        [0.71078148, 2.63323323, 1.92245175, 3.54548846, 2.63323323],
        [0.26561909, 2.41722084, 0.97640056, 2.63323323, 2.78510029]])

In [6]:
# read off diagonal values
np.diagonal(mtx_expA)

array([1.59268734, 2.78510029, 2.30346881, 3.54548846, 2.78510029])

In [7]:
print(np.trace(mtx_expA))

13.01184518629529


In [8]:
norm(A.todense())

3.1622776601683795

In [9]:
# find central-most node and Estrada index
print('Central-most node is',np.argmax(np.diagonal(mtx_expA))+1,'which has subgraph centrality',np.max(np.diagonal(mtx_expA)),
      '.\nThe Estrada Index of A is',np.trace(mtx_expA))

Central-most node is 4 which has subgraph centrality 3.545488464905249 .
The Estrada Index of A is 13.01184518629529


In [10]:
# Arnoldi algorithm

# matrix A for which exp(A)b is of interest, n x n

# b initial vector to be used, length n

# m, the produced Krylov subspace will have dimension m


def arnoldi_iteration(A, m: int, b = np.ones(A.shape[0])): # b default to be 1_n
    
    n = A.shape[0]
    h = np.zeros((m + 1, m)) # to become the m x m upper Hessenberg matrix consisting of the coefficients h_ij
    V = np.zeros((n, m + 1)) # to become the orthonormal basis V_m = [v_1, v_2, ..., v_m]
    v = b / norm(b) # makes v a unit 2-norm vector    
    V[:, 0] = v # use v as the first Krylov vector
    
    for j in range(m):
        w = A @ v  # compute candidate vector
        
        for i in range(j + 1):
            h[i, j] = V[:,i] @ w # h_ij-th element is product of v_i and w
            w = w - h[i, j] * V[:, i] # modified Gram-Schmidt
            
        h[j + 1, j] = norm(w)
        
        zero = 1e-12 # small value used as h_ij = 0 threshold
        if h[j + 1, j] > zero: # if nonzero add v to the basis
            v = w / h[j + 1, j]
            V[:, i + 1] = v
        else: 
            return V, h 
        # print('step',j,'out of',m) # to check how far along algorithm is for larger m
    return V, h
            
            

In [20]:
# Arnoldi approximation

# exp(A)v ~ beta x V_m x exp(H_m) x e_1 (m-space 1st unit vector [1, 0, 0,..., 0])
m = 100

# get results 

Vm1, Hm1 = arnoldi_iteration(A, m)

Vm, Hm = sparse.csc_matrix(Vm1[:,0:m]), sparse.csc_matrix(Hm1[0:m,0:m])

# checks

print("check Arnoldi relation", A @ Vm - Vm @ Hm) # should be zero matrix (okay)

orthog = Vm.transpose() @ Vm
print("check orthogonality", orthog.todense()) # should be identity (okay)


# get beta = ||v||_2

b = np.ones(A.shape[0])
beta = norm(b)

# get unit vector

e1 = np.zeros((m,1))
e1[0] = 1
e1 = sparse.csc_matrix(e1)

check Arnoldi relation   (2, 3)	-2.220446049250313e-16
  (3, 3)	-1.1102230246251565e-16
  (4, 3)	-2.220446049250313e-16
  (0, 3)	-1.1102230246251565e-16
  (1, 3)	-2.220446049250313e-16
check orthogonality [[1.00000000e+00 5.55111512e-17 1.38777878e-16 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.55111512e-17 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.38777878e-16 0.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [21]:
# check Vm
np.around(sparse.csc_matrix.toarray(Vm), decimals = 3)

array([[ 0.447, -0.707, -0.365,  0.408,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.447,  0.   ,  0.548, -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,

In [22]:
# check Hm
np.around(sparse.csc_matrix.toarray(Hm), decimals = 3)

array([[ 2.   ,  0.632,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.632, -0.   ,  0.775, ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.775,  0.333, ...,  0.   ,  0.   ,  0.   ],
       ...,
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ]])

In [23]:
np.around(sparse.csc_matrix.toarray(orthog), decimals = 3)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
# arnoldi approximation
X = beta * Vm @ expm(Hm) @ e1
print('Arnoldi approximation\n',X.todense())

Arnoldi approximation
 [[ 4.22592057]
 [ 9.07757401]
 [ 7.56993527]
 [11.44518816]
 [ 9.07757401]]


In [25]:
# expAb calulated directly
exact = expm(A) @ sparse.csc_matrix(b.reshape(5,1))
print('Exact',exact.todense())

Exact [[ 4.22592057]
 [ 9.07757401]
 [ 7.56993527]
 [11.44518816]
 [ 9.07757401]]


In [26]:
print('Arnoldi approximation\n',np.around(sparse.csc_matrix.toarray(X), 3))

Arnoldi approximation
 [[ 4.226]
 [ 9.078]
 [ 7.57 ]
 [11.445]
 [ 9.078]]


In [27]:
print('Exact',np.around(sparse.csc_matrix.toarray(exact), 3))

Exact [[ 4.226]
 [ 9.078]
 [ 7.57 ]
 [11.445]
 [ 9.078]]


In [19]:
# arnoldi_iteration(A, b, m)
# Vm, Hm = arnoldi_iteration(A, b, m)
# e_1 = np.zeros((m,1))
# e_1[1] = 1
# X = beta * Vm[:,0:m] @ expm(Hm[0:m,0:m]) @ e_1